In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
import os 
from dotenv import load_dotenv
from time import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
loader = TextLoader('Woodcraft and Camping.txt', encoding='utf-8')
documents = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
texts = text_splitter.split_documents(documents)

In [4]:
len(texts)

156

In [5]:
texts[10]

Document(page_content="Chapter II\n\n     KNAPSACK, HATCHET, KNIVES, TINWARE, RODS, FISHING TACKLE,\n     DITTY-BAG\n\n\nThe clothing, blanket-bag and shelter-cloth are all that need be\ndescribed in that line. The next articles that I look after are knapsack\n(or pack basket), rod with reel, lines, flies, hooks, and all my fishing\ngear, pocket-axe, knives and tinware. Firstly, the knapsack; as you are\napt to carry it a great many miles, it is well to have it right, and\neasy-fitting at the start. Don't be induced to carry a pack basket. I am\naware that it is in high favor all through the Northern Wilderness, and\nis also much used in other localities where guides and sportsmen most do\ncongregate. But I do not like it. I admit that it will carry a loaf of\nbread, with tea, sugar, etc., without jamming; that bottles, crockery,\nand other fragile duffle is safer from breakage than in an oil-cloth\nknapsack. But it is by no means water-proof in a rain or a splashing\nhead sea, is more

In [6]:
_ = load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

llm=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta", 
    model_kwargs={"temperature":0.2, "max_length":256},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
    )

In [7]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="BAAI/bge-base-en-v1.5",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
%%time
persist_directory = 'db_HuggingFace'

embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: total: 3.28 s
Wall time: 6 s


In [9]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
docs = retriever.get_relevant_documents("Woodcraft and Camping.txt')")

In [10]:
len(docs)

2

In [11]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [12]:
def process_llm_response(qa_chain, query):
    print(f"Query: {query}\n")
    time_1 = time()
    llm_response = qa_chain(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print(f"\nResult:", llm_response['result'])
    print(f"\nmetadata:", llm_response['source_documents'][0].metadata)

In [13]:
query = "What do we do if we get lost?"
process_llm_response(qa_chain, query)
#wrong metadata

Query: What do we do if we get lost?

Inference time: 1.581 sec.

Result:  If we get lost, we should stop and look around to see if we can find any landmarks or signs that might help us figure out where we are. If we still can't find our way, we should try to retrace our steps and follow the route we came in on. If that doesn't work, we should try to find a nearby town or city and ask for directions. It's always better to be safe than sorry when it comes to getting lost, so we should never hesitate to ask for help if we need it.

metadata: {'source': 'The existence of Gods.txt'}


In [14]:
query = "What is the importance of making the camp-fire?"
process_llm_response(qa_chain, query)

Query: What is the importance of making the camp-fire?

Inference time: 0.478 sec.

Result:  The camp-fire is important because it provides warmth and light for the camp, and it can also be used for cooking and boiling water for drinking. Making a proper camp-fire is crucial to ensure safety and prevent accidents. It should be built in a clear area away from flammable materials, and proper precautions should be taken to prevent it from spreading. Additionally, the fire should be properly extinguished before leaving the campsite to prevent wildfires.

metadata: {'source': 'Woodcraft and Camping.txt'}


In [15]:
query = "List out 5 things we need to bring for camping"
process_llm_response(qa_chain, query)

Query: List out 5 things we need to bring for camping

Inference time: 0.439 sec.

Result:  Some things we might need to bring for camping include a tent, sleeping bags, a camping stove for cooking, a cooler for storing food, and a first aid kit in case of any accidents. Do you have any other suggestions?

metadata: {'source': 'Woodcraft and Camping.txt'}


In [16]:
query = "How camp cooking is usually done? "
process_llm_response(qa_chain, query)

Query: How camp cooking is usually done? 

Inference time: 0.464 sec.

Result:  Camp cooking typically involves cooking meals outdoors using portable stoves, grills, or open fires. Food is often prepared in advance and packed in coolers or insulated bags to keep it fresh. Simple ingredients and easy-to-prepare meals are commonly used to minimize the amount of equipment and cleanup required. Some common camping foods include hot dogs, hamburgers, foil packet meals, and pre-packaged camping meals.

metadata: {'source': 'Woodcraft and Camping.txt'}


In [17]:
query = "2 Simple receipts for camp cooking"
process_llm_response(qa_chain, query)

Query: 2 Simple receipts for camp cooking

Inference time: 0.543 sec.

Result:  1. Campfire Bannock:
Ingredients:
- 2 cups all-purpose flour
- 1 tbsp baking powder
- 1 tsp salt
- 1/4 cup vegetable oil
- 3/4 cup water

Instructions:
1. In a large bowl, mix together the flour, baking powder, and salt.
2. Add the vegetable oil and water, and stir until a dough forms.
3. Knead the dough on a floured surface for 1-2 minutes.
4. Roll the dough out into a circle about 1/4 inch thick.
5. Place the circle directly on the campfire grate or a campfire-safe skillet.
6. Cook for 2-3 minutes on each side, or until golden brown.

2. Campfire Pancakes:
Ingredients:
- 1 cup all-purpose flour
- 2 tbsp sugar
- 2 tsp baking powder
- 1/2 tsp salt
- 1 cup milk
- 1 egg
- 2 tbsp vegetable oil

Instructions:
1. In a large bowl, mix together the flour, sugar, baking powder, and salt.
2. Add the milk, egg, and vegetable oil, and stir until just combined.
3. Heat a campfire-safe skillet over the campfire.
4. Pour

In [18]:
query = "Reasons for Preferring the Clinker-Built Cedar?"
process_llm_response(qa_chain, query)
#wrong metadata

Query: Reasons for Preferring the Clinker-Built Cedar?

Inference time: 0.484 sec.

Result:  The author prefers the clinker-built cedar because it is light, strong, and durable, and it can be built into any part of the ground that the builder pleases. The author also mentions that some people still prefer this type of boat despite its perceived flaws, such as being "merciful, good, wise, faithful." However, the author does not provide specific reasons for these preferences.

metadata: {'source': 'The existence of Gods.txt'}


In [19]:
query = "How to make a camp?"
process_llm_response(qa_chain, query)
#wrong metadata

Query: How to make a camp?

Inference time: 0.573 sec.

Result:  To make a camp, you will need a tent, sleeping bags, a camping stove, a cooler for food, a first aid kit, and a map of the area. Follow the directions provided to set up your tent and arrange your gear. Be sure to properly dispose of any waste and leave the area as you found it. If you encounter any difficult situations, consult your map and use your judgment to make informed decisions. Remember to always prioritize safety and respect for the environment.

metadata: {'source': 'The existence of Gods.txt'}


In [20]:
query = "How to make a camp fire?"
process_llm_response(qa_chain, query)

Query: How to make a camp fire?

Inference time: 0.435 sec.

Result:  To make a camp fire, first find a level spot and clear away any dry grass or leaves. Then gather some small sticks and twigs and arrange them in a teepee shape. Light a match and ignite the sticks, adding larger pieces of wood as the fire grows. Be sure to keep the fire contained and never leave it unattended.

metadata: {'source': 'Woodcraft and Camping.txt'}


In [ ]:
query = "Where to go for an Outing?"
process_llm_response(qa_chain, query)

Query: Where to go for an Outing?



In [ ]:
query = "Give me an example of a easily made and very comfortable camp"
process_llm_response(qa_chain, query)

In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)